In [1]:
import cv2 as cv
import torch
import numpy as np
from tqdm import tqdm
import datetime

/home/user/anaconda3/envs/harshita_env/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def preprocess_image(orig_image):
    #orig_image=cv.imread(orig_image_path)
    img = cv.cvtColor(orig_image, cv.COLOR_BGR2RGB).astype(np.float32)

    img=cv.resize(img,(224,224))
    img=img/255.0

    img = torch.tensor(img).permute(2,0,1)
    img=img.float()
    return img

In [98]:
video_file_paths=['/home/user/Jeba/CAREADD_PLAYvideos/recordings/Annotated_New/TD group/21_01_13_01_TD/Ch1_20210113120050.mp4',
'/home/user/Jeba/CAREADD_PLAYvideos/recordings/Annotated_New/TD group/21_01_13_01_TD/Ch1_20210113121804.mp4']

frame_skip=60
i=0
frame_count=0
test_images=[]
for video_file_path in video_file_paths:
    cam = cv.VideoCapture(video_file_path)
    cam.get(cv.CAP_PROP_FPS)
    
    while(True):
                
                # reading from frame
                ret,frame = cam.read()
                #print(ret)
                if ret:
                    # if video is still left continue creating images
                    
                    if i > frame_skip - 1:
                        frame_count += 1
                        #print(video_file+'\n'+video_file[:-3])
                        #name = './Test_Results/Video Results/'+video_file_name.split('/')[-1][:-4] +'_frame_'+ str(frame_count)+'.jpg'
                        #print(name)
                        #cv.imwrite(name, frame)
                        #print('Done')
                        test_images.append(preprocess_image(frame))
                        i = 0
                        continue
                    i += 1
                    
                else:
                    break
        
        # Release all space and windows once done
    cam.release()
    cv.destroyAllWindows()


In [99]:
len(test_images)

859

In [100]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.ops import nms

device = 'cuda' if torch.cuda.is_available() else 'cpu'

def get_model():
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model


def decode_output(output):
    'convert tensors to numpy arrays'
    bbs = output['boxes'].cpu().detach().numpy().astype(np.uint16)
    labels = np.array([target2label[i] for i in output['labels'].cpu().detach().numpy()])
    confs = output['scores'].cpu().detach().numpy()
    ixs = nms(torch.tensor(bbs.astype(np.float32)), torch.tensor(confs), 0.05)
    bbs, confs, labels = [tensor[ixs] for tensor in [bbs, confs, labels]]

    if len(ixs) == 1:
        bbs, confs, labels = [np.array([tensor]) for tensor in [bbs, confs, labels]]
    return bbs.tolist(), confs.tolist(), labels.tolist()


def collect_results(chunks,model):
    all_bbs=[]
    all_confs=[]
    all_labels=[]
    for chunk in tqdm(chunks):
        outputs=model(chunk)
        
        for output in outputs:
            bbs,confs,labels=decode_output(output)
            temp_bbs=[]
            temp_confs=[]
            temp_labels=[]
            for b,c,l in zip(bbs,confs,labels):
                if c>=0.4:
                    temp_bbs.append(b)
                    temp_confs.append(c)
                    temp_labels.append(l)
            all_bbs.append(temp_bbs)
            all_confs.append(temp_confs)
            all_labels.append(temp_labels)
    return all_bbs,all_confs,all_labels

In [101]:
label2target={
    'BLOCKS': 1,
    'RATTLE': 2,
    'BALL': 3,
    'BANANA': 4,
    'TOYANIMAL': 5,
    'DOLL': 6,
    'CAR': 7,
    'BELL': 8,
    'STICKS': 9,
    'TOYPHONE': 10,
    'SOFTTOY': 11,
    'background': 0
}

target2label={
    1: 'BLOCKS',
    2: 'RATTLE',
    3: 'BALL',
    4: 'BANANA',
    5: 'TOYANIMAL',
    6: 'DOLL',
    7: 'CAR',
    8: 'BELL',
    9: 'STICKS',
    10: 'TOYPHONE',
    11: 'SOFTTOY',
    0: 'background'
}

background_class = label2target['background']
num_classes = len(label2target)

In [102]:
model= get_model()
model.load_state_dict(torch.load('Faster_RCNN_Trained_1500'))
model.eval()
chunks = [test_images[x:x+8] for x in range(0, len(test_images), 8)]
all_bbs,all_confs,all_labels=collect_results(chunks,model)

100%|██████████| 108/108 [16:04<00:00,  8.93s/it]


In [103]:
import pickle
with open("Results_Objects/all_labels_40.pickle", 'wb') as outp:
        pickle.dump(all_labels, outp, pickle.HIGHEST_PROTOCOL)

with open("Results_Objects/all_confs_40.pickle", 'wb') as outp:
        pickle.dump(all_confs, outp, pickle.HIGHEST_PROTOCOL)

with open("Results_Objects/all_bbs_40.pickle", 'wb') as outp: 
        pickle.dump(all_bbs, outp, pickle.HIGHEST_PROTOCOL)


# with open(PIK, "rb") as f:
#     print pickle.load(f)

In [125]:
threshold=2
threshold_toyanimal=1
threshold_fp2=15
results={}
for t,label_conf in enumerate(zip(all_labels,all_confs)):
    if ('Idle' not in results.keys()) and ('BALL' in label_conf[0] or 'RATTLE' in label_conf[0] or 'BLOCKS' in label_conf[0]):
            print('Inside Idle')
            results['Idle']=str(datetime.timedelta(seconds=t*(60/29.9)))
            continue
    print(t,label_conf)
    if ('FP1' not in results.keys()) and ('BALL' in label_conf[0] and len(label_conf[0])==1):
            print('Inside FP1 '+str(t))
            temp=0
            for i in range(threshold):
                if t+i+1 < len(all_labels) and 'BALL' in all_labels[t+i+1] and len(all_labels[t+i+1])==1:
                    print(t+i+1,all_labels[t+i+1])
                    temp=temp+1
            if temp==threshold:
                results['FP1']=str(datetime.timedelta(seconds=t*(60/29.9)))
                print('FP1 done!')
                continue
    if ('BALL' not in results.keys()) and ('RATTLE' in label_conf[0] and len(label_conf[0])==1):
            print('Inside BALL '+str(t))
            temp=0
            for i in range(threshold):
                if t+i+1 < len(all_labels) and 'RATTLE' in all_labels[t+i+1] and len(all_labels[t+i+1])==1:
                    print(t+i+1,all_labels[t+i+1])
                    temp=temp+1
            if temp==threshold:
                results['BALL']=str(datetime.timedelta(seconds=t*(60/29.9)))
                print('BALL done!')
                continue
    if ('RATTLE' not in results.keys()) and ('BLOCKS' in label_conf[0] and len(set(label_conf[0]))==1):
            print('Inside RATTLE '+str(t))
            temp=0
            for i in range(threshold):
                if t+i+1 < len(all_labels) and 'BLOCKS' in all_labels[t+i+1] and len(set(all_labels[t+i+1]))==1:
                    print(t+i+1,all_labels[t+i+1])
                    temp=temp+1
            if temp==threshold:
                results['RATTLE']=str(datetime.timedelta(seconds=t*(60/29.9)))
                print('RATTLE done!')
                continue
    if ('BLOCKS' not in results.keys()) and ('BANANA' in label_conf[0] and len(label_conf[0])==1):
            print('Inside BLOCKS '+str(t))
            temp=0
            for i in range(threshold):
                if t+i+1 < len(all_labels) and 'BANANA' in all_labels[t+i+1] and len(all_labels[t+i+1])==1:
                    print(t+i+1,all_labels[t+i+1])
                    temp=temp+1
            if temp==threshold:
                results['BLOCKS']=str(datetime.timedelta(seconds=t*(60/29.9)))
                print('BLOCKS done!')
                continue
    if ('BANANA' not in results.keys()) and ('TOYANIMAL' in label_conf[0] and len(label_conf[0])==1):
            print('Inside BANANA '+str(t))
            temp=0
            for i in range(threshold_toyanimal):
                if t+i+1 < len(all_labels) and 'TOYANIMAL' in all_labels[t+i+1] and len(all_labels[t+i+1])==1:
                    print(t+i+1,all_labels[t+i+1])
                    temp=temp+1
            if temp==threshold_toyanimal:
                results['BANANA']=str(datetime.timedelta(seconds=t*(60/29.9)))
                print('BANANA done!')
                continue

    if ('TOYANIMAL' not in results.keys()) and (('BALL' in label_conf[0] or 'RATTLE' in label_conf[0]) and 'BLOCKS' in label_conf[0] and ('BANANA' in label_conf[0] or 'TOYANIMAL' in label_conf[0])):
            print('Inside TOYANIMAL '+str(t))
            print(t,all_labels[t])
            temp=0
            for i in range(threshold):
                if t+i+1 < len(all_labels) and (('BALL' in label_conf[0] or 'RATTLE' in label_conf[0]) and 'BLOCKS' in label_conf[0] and ('BANANA' in label_conf[0] or 'TOYANIMAL' in label_conf[0])):
                    print('Inside loop',t+i+1,all_labels[t+i+1])
                    temp=temp+1
            print(temp)
            if temp==threshold:
                results['TOYANIMAL']=str(datetime.timedelta(seconds=t*(60/29.9)))
                print('TOYANIMAL done!')
                continue

    if ('FP2' not in results.keys()) and ('STICKS' in label_conf[0] and len(set(label_conf[0]))==1):
            print('Inside FP2 '+str(t))
            temp=0
            for i in range(threshold):
                if t+i+1 < len(all_labels) and 'STICKS' in all_labels[t+i+1] and len(set(all_labels[t+i+1]))==1:
                    print(t+i+1,all_labels[t+i+1])
                    temp=temp+1
            if temp==threshold:
                results['FP2']=str(datetime.timedelta(seconds=t*(60/29.9)))
                print('FP2 done!')
                continue

    if ('STICKS' not in results.keys()) and ('DOLL' in label_conf[0] and len(label_conf[0])==1):
            print('Inside STICKS '+str(t))
            temp=0
            for i in range(threshold_toyanimal):
                if t+i+1 < len(all_labels) and 'DOLL' in all_labels[t+i+1] and len(all_labels[t+i+1])==1:
                    print(t+i+1,all_labels[t+i+1])
                    temp=temp+1
            if temp==threshold_toyanimal:
                results['STICKS']=str(datetime.timedelta(seconds=t*(60/29.9)))
                print('STICKS done!')
                continue

    if ('DOLL' not in results.keys()) and ('DOLL' in label_conf[0] and 'TOYANIMAL' in label_conf[0] ):  ##confusion among toyphone, toy animal & toy car
            print('Inside DOLL '+str(t))
            temp=0
            for i in range(threshold):
                if t+i+1 < len(all_labels) and ('DOLL' in all_labels[i+t+1] and 'TOYANIMAL' in all_labels[i+t+1] ) :
                    print(t+i+1,all_labels[t+i+1])
                    temp=temp+1
            if temp==threshold:
                results['DOLL']=str(datetime.timedelta(seconds=t*(60/29.9)))
                print('DOLL done!')
                continue
            


0 ([], [])
1 ([], [])
2 ([], [])
3 ([], [])
4 ([], [])
5 ([], [])
6 ([], [])
7 ([], [])
8 ([], [])
9 ([], [])
10 ([], [])
11 ([], [])
12 ([], [])
13 ([], [])
14 ([], [])
15 ([], [])
16 ([], [])
17 ([], [])
18 ([], [])
19 ([], [])
20 ([], [])
21 ([], [])
22 ([], [])
23 ([], [])
24 ([], [])
25 ([], [])
26 ([], [])
27 ([], [])
28 ([], [])
29 ([], [])
30 ([], [])
31 ([], [])
32 ([], [])
33 ([], [])
34 ([], [])
35 ([], [])
36 ([], [])
37 ([], [])
38 ([], [])
39 ([], [])
40 ([], [])
41 ([], [])
42 ([], [])
43 ([], [])
44 ([], [])
45 ([], [])
46 ([], [])
47 ([], [])
48 ([], [])
49 ([], [])
50 ([], [])
51 ([], [])
52 ([], [])
53 ([], [])
54 ([], [])
55 ([], [])
56 ([], [])
57 ([], [])
58 ([], [])
59 ([], [])
60 ([], [])
61 ([], [])
62 ([], [])
63 ([], [])
64 ([], [])
65 ([], [])
66 ([], [])
67 (['DOLL'], [0.48913416266441345])
Inside STICKS 67
68 ([], [])
69 ([], [])
70 ([], [])
71 ([], [])
72 ([], [])
73 ([], [])
74 ([], [])
75 ([], [])
76 ([], [])
77 ([], [])
78 ([], [])
79 ([], [])
80 ([], 

In [126]:
results

{'Idle': '0:03:42.742475',
 'TOYANIMAL': '0:03:46.755853',
 'FP1': '0:09:57.993311',
 'BALL': '0:11:58.394649',
 'RATTLE': '0:12:20.468227',
 'BLOCKS': '0:14:12.842809',
 'BANANA': '0:14:48.963211',
 'FP2': '0:18:47.759197',
 'STICKS': '0:19:47.959866',
 'DOLL': '0:24:02.809365'}